In [2]:
import time as tm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopy as geopy
from geopy.geocoders import Nominatim
from folium.plugins import HeatMap
import folium # map rendering library
import branca.colormap as cm
import re

print(f"Pandas version: {pd.__version__}")
print(f"geopy version: {geopy.__version__}")

Pandas version: 1.2.4
geopy version: 2.2.0


In [3]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [4]:
mo_geo = r'../data/_mo.geojson'

In [5]:
#df = pd.read_csv('Moscow_districts.csv')
df = pd.read_csv('../data/Moscow_districts(old_borders).csv')
df

,Name,Name_rus,Population
0,Aeroport (Аэропорт)74775,NaN,NaN
1,Begovoy (Беговой)44385,NaN,NaN
2,Beskudnikovsky (Бескудниковский)74790,NaN,NaN
3,Dmitrovsky (Дмитровский)88931,NaN,NaN
4,Golovinsky (Головинский)102160,NaN,NaN
...,...,...,...
113,Pokrovskoye-Streshnevo (Покровское-Стрешнево)4...,NaN,NaN
114,Severnoe Tushino (Северное Тушино)138533,NaN,NaN
115,Shchukino (Щукино)89454,NaN,NaN
116,Strogino (Строгино)124149,NaN,NaN


In [6]:
df['Population'] = df['Name'].str.extract(r'(\d+)', expand=False).astype(int)
df['Name_rus']   = df['Name'].str.extract(r'([А-Яа-яЁё\-\s]+[А-Яа-яЁё])\)', expand=False)
df['Name'] = df.apply(lambda x : re.search(r'([a-zA-Z\-\s]+[a-zA-Z])', x['Name']).group(0), axis=1)
df

,Name,Name_rus,Population
0,Aeroport,Аэропорт,74775
1,Begovoy,Беговой,44385
2,Beskudnikovsky,Бескудниковский,74790
3,Dmitrovsky,Дмитровский,88931
4,Golovinsky,Головинский,102160
...,...,...,...
113,Pokrovskoye-Streshnevo,Покровское-Стрешнево,46707
114,Severnoe Tushino,Северное Тушино,138533
115,Shchukino,Щукино,89454
116,Strogino,Строгино,124149


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        118 non-null    object
 1   Name_rus    118 non-null    object
 2   Population  118 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 2.9+ KB


In [10]:
df_objects = pd.read_csv('../data/el_infostructure.csv')

In [11]:
df_objects

,Unnamed: 0,id,class,tags,name_ru,operator,tags_ru,opening_hours,class_ru,addr_region,addr_district,addr_city,addr_full_name,index_name,addr_region_id,addr_district_id,addr_city_id,addr_house_id,addr_in_id,addr_in_type,brand,phone,fax,website,addr_village,addr_street,addr_house,c_geom,addr_country,email,description,wikipedia,osm_id,lat,lon,nclass
0,0,194014721,substation,{},NaN,NaN,"{'Напряжения': None, 'Газовая изоляция': 'неиз...",NaN,Подстанция,Москва,NaN,NaN,"Российская Федерация, Москва","Подстанция , , Москва, Российская Федерация,",787547493,NaN,NaN,NaN,787547493,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0101000020E610000006789EBA27D14240BB959C016DE4...,Российская Федерация,NaN,NaN,NaN,w588157494,55.785,37.634,substation
1,1,194014711,substation,{},NaN,NaN,"{'Напряжения': None, 'Газовая изоляция': 'неиз...",NaN,Подстанция,Москва,NaN,NaN,"Российская Федерация, Москва","Подстанция , , Москва, Российская Федерация,",787547493,NaN,NaN,NaN,787547493,10,NaN,NaN,NaN,NaN,NaN,улица Академика Петровского,10 с4,0101000020E6100000F8B8BB83A7CD42403395E50A21DC...,Российская Федерация,NaN,NaN,NaN,w587925551,55.720,37.607,substation
2,2,193841447,substation,{},NaN,NaN,"{'Напряжения': None, 'Газовая изоляция': 'неиз...",NaN,Подстанция,Московская область,NaN,NaN,"Российская Федерация, Московская область","Подстанция , , Московская область, Российская...",787547503,NaN,NaN,NaN,787547503,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0101000020E61000006A9DBD529BEE4240F83A9092DFDF...,Российская Федерация,NaN,NaN,NaN,w89647866,55.749,37.864,substation
3,3,193841032,substation,{},NaN,NaN,"{'Напряжения': None, 'Газовая изоляция': 'неиз...",NaN,Подстанция,Московская область,NaN,NaN,"Российская Федерация, Московская область","Подстанция , , Московская область, Российская...",787547503,NaN,NaN,NaN,787547503,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0101000020E6100000D6A0D4038BF24240FA6A589D66E5...,Российская Федерация,NaN,NaN,NaN,w89281891,55.792,37.895,substation
4,4,193841446,substation,{},NaN,NaN,"{'Напряжения': None, 'Газовая изоляция': 'неиз...",NaN,Подстанция,Московская область,NaN,NaN,"Российская Федерация, Московская область","Подстанция , , Московская область, Российская...",787547503,NaN,NaN,NaN,787547503,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0101000020E61000009BBDD820B8EE42403F834D9DC7DF...,Российская Федерация,NaN,NaN,NaN,w89647859,55.748,37.865,substation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,194162573,power_plant,{},ГЭС-1 им. П. Г. Смидовича,ПАО «Мосэнерго»,"{'Выработка горячего воздуха': None, 'Выработк...",NaN,Электростанция,Москва,NaN,NaN,"Российская Федерация, Москва","Электростанция , ГЭС-1 им. П. Г. Смидовича , М...",787547493,NaN,NaN,NaN,787547493,10,NaN,NaN,NaN,http://www.mosenergo.ru/docs/info/386.aspx,NaN,NaN,NaN,0101000020E6100000E6B6BDF2C0D04240DD1D85D2B9DF...,Российская Федерация,NaN,NaN,ru:ГЭС-1,r9975357,55.748,37.631,power_plant
996,996,194167338,power_generator,{},NaN,NaN,"{'Выработка горячего воздуха': None, 'Выработк...",NaN,Генератор,Москва,NaN,NaN,"Российская Федерация, Москва","Генератор , , Москва, Российская Федерация,",787547493,NaN,NaN,NaN,787547493,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0101000020E6100000CB9D9960B8D84240760E65A80ADF...,Российская Федерация,NaN,NaN,NaN,w835481732,55.743,37.693,power_generator
997,997,194161352,power_plant,{},Карамышевская ГЭС,NaN,"{'Выработка горячего воздуха': None, 'Выработк...",NaN,Электростанция,Москва,NaN,NaN,"Российская Федерация, Москва","Электростанция , Карамышевская ГЭС , Москва, Р...",787547493,NaN,NaN,NaN,787547493,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0101000020E610000017486A7A9DBB4240EA43787C9EE1...,Российская Федерация,NaN,NaN,NaN,w846683651,55.763,37.466,power_plant
998,998,194162584,power_plant,{},"ТЭС ""Международная""",NaN,"{'Выработка горячего воздуха': None, 'Выработк...",NaN,Электростанция,Москва,NaN,NaN,"Российская Федерация, Москва","Электростанция , ТЭС ""Международная"" , Москва,...",78754749

In [15]:
df.columns

Index(['Name', 'Name_rus', 'Population'], dtype='object')

In [25]:
geolocator = Nominatim(user_agent="ny_explorer")
address = 'Moscow'
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

moscow_map = folium.Map(location=[latitude, longitude], zoom_start=10)

HeatMap(df_objects[['lat', 'lon']], radius=10, blur=8).add_to(moscow_map)
    
moscow_map

The geograpical coordinate of Moscow are 55.7504461, 37.6174943.


In [40]:
df_cinema_mall = pd.read_excel('../data/cinema_and_mall.xlsx')
df_cinema_mall

,address,lat,lon,name,rubric
0,"Россия, Москва, Ходынский бульвар, 4",55.790,37.531,Авиапарк,Торговый центр
1,"Россия, Москва, МКАД, 14-й километр, 30",55.655,37.830,Садовод,Торговый центр
2,"Россия, Москва, проспект Мира, 211к2",55.846,37.662,Европолис,Торговый центр
3,"Россия, Москва, площадь Киевского Вокзала, 2",55.744,37.566,Европейский,Торговый центр
4,"Россия, Москва, Кировоградская улица, 13А",55.612,37.607,Columbus,Торговый центр
...,...,...,...,...,...
601,"Россия, Москва, Киевская улица, 2, стр. 5",55.744,37.563,ЦУМ дисконт,Торговый центр
602,"Россия, Москва, Фестивальная улица, 28, стр. 1",55.861,37.503,Ховринский хозяин,Торговый центр
603,"Россия, Москва, проспект Мира, 119, стр. 18",55.832,37.625,Белорусский торгово-выставочный и информационн...,Торговый центр
604,"Россия, Москва, улица Шаболовка, 34, стр. 3",55.718,37.607,Шаболовский,Торговый центр


In [41]:
df_parking = pd.read_excel('../data/parking.xlsx')
df_parking

,address,lat,lon,name
0,"Россия, Москва, Пресненская набережная, 2",55.748,37.538,"ТРЦ Афимолл Сити, подземная парковка"
1,"Россия, Москва, Ленинградское шоссе, 16Ас7",55.825,37.497,Парковка ТЦ Метрополис
2,"Россия, Москва, парк Зарядье",55.751,37.626,Подземная парковка
3,"Россия, Москва, Театральный проезд, 5, стр. 1",55.760,37.624,Парковка Центрального Детского Магазина
4,"Россия, Москва, улица Ильинка, 3/8с1",55.754,37.624,Парковка ГУМ
...,...,...,...,...
995,"Москва, Кленовый бульвар, 25вл1",55.674,37.683,Автостоянка № 279
996,"Россия, Москва, Долгоруковская улица, 15с1",55.774,37.604,Парковка
997,"Россия, Москва, Подкопаевский переулок, 4",55.754,37.642,Автомобильная парковка
998,"Россия, Москва, Инновационный центр Сколково, ...",55.690,37.344,Парковка Технопарк


In [54]:
geolocator = Nominatim(user_agent="ny_explorer")
address = 'Moscow'
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

moscow_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for i in range(len(df_cinema_mall)):
    folium.Circle(
        radius=3,
        location=[df_cinema_mall['lat'][i], df_cinema_mall['lon'][i]],
        popup=df_cinema_mall.rubric[i],
        color='blue',
        fill=True,
    ).add_to(moscow_map)
    
for i in range(len(df_parking)):
    folium.Circle(
        radius=3,
        location=[df_parking['lat'][i], df_parking['lon'][i]],
        popup=df_parking.name[i],
        color='red',
        fill=True,
    ).add_to(moscow_map)

HeatMap(df_objects[['lat', 'lon']], radius=20, blur=20).add_to(moscow_map)
    
moscow_map

The geograpical coordinate of Moscow are 55.7504461, 37.6174943.


In [55]:
moscow_map.save('heatmap.html')